In [ ]:
#ORIGINAL CODE: https://github.com/NeuroanatomyAndConnectivity/gradient_analysis

In [1]:
import numpy as np
#git clone --branch enh/cifti2 https://github.com/satra/nibabel.git
#import nibabel as nib 
#from sklearn.metrics import pairwise_distances

In [2]:
module('list')
module('load','nibabel')
module('unload',"nibabel")

Currently Loaded Modulefiles:
  1) rvs/1.0              2) anaconda/3/2019.03


In [ ]:
import nibabel as nib
import os
import numpy as np
#file_path = "E:\hcp data\HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn"
#example_filename = os.path.join(file_path, "HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii")
#img = nib.load(example_filename)
img = nib.load("HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii")

In [ ]:
import sys
data_tanh = img.get_data()
data_tanh = np.tanh(data_tanh)


In [ ]:
np.save('data.npy', data_tanh) #data is np.tanh(img)

In [ ]:
dcon = data_tanh

In [ ]:
import nibabel as nib
# Load data and Fisher's z-to-r transform
#dcon = np.tanh(nib.load('HCP_S1200_1003_rfMRI_MSMAll_groupPCA_d4500ROW_zcorr.dconn.nii'))
#dcon = np.tanh(data)
# Get number of nodes
N = dcon.shape[0]
dcon.shape
# Generate percentile thresholds for 90th percentile
#perc = np.array([np.percentile(np.tanh(x), 90) for x in data])
#perc = np.array([np.percentileqq(np.tanh(data), 90, axis = 0)])

"""
per = []
for i,x in enumerate(data):
    p = np.percentile(np.tanh(x), 90)
    per.append(p)
    print(i)
    del p"""
'''
per2 = []
for k in range(9):
    x = data[k*1000:(k+1)*1000, :]
    p = [np.percentile(np.tanh(x), 90, axis = 0)  for x in data]
    print(p)
    per2.append(p)
    print(k)
    del x

x = data[9000:, :]
p = np.percentile(np.tanh(x), 90, axis = 0)
del x
per2.append(p)

# Threshold each row of the matrix by setting values below 90th percentile to 0
for i in range(dcon.shape[0]):
  #print("Row %d" % i)
  data[i, dcon[i,:] < perc[i]] = 0'''

In [ ]:
#perc = np.array([np.percentile(np.tanh(x), 90) for x in dcon])


In [ ]:
#np.save("perc",perc)

In [ ]:
perc = np.load('perc.npy')

In [ ]:
dcon = perc
N = dcon.shape[0]
dcon.shape

In [ ]:
import numpy as np
dcon = np.load('data.npy')

In [6]:
N = dcon.shape[0]
dcon.shape

(91282, 91282)

In [7]:
print("Minimum value is %f" % dcon.min())


Minimum value is -0.557308


In [6]:
dcon[dcon < 0] = 0

In [7]:
# Check for minimum value
# print("Minimum value is %f" % dcon.min())

# The negative values are very small, but we need to know how many nodes have negative values
# Count negative values per row
# neg_values = np.array([sum(dcon[i,:] < 0) for i in range(N)])
# print("Negative values occur in %d rows" % sum(neg_values > 0))

# Since there are only 23 vertices with total of 5000 very small negative values, we set these to zero
# dcon[dcon < 0] = 0

In [ ]:
np.save('dcon_nonzero.npy', dcon)

In [4]:
import numpy as np
dcon = np.load('dcon_nonzero.npy')

In [5]:
from sklearn.metrics.pairwise import pairwise_distances_chunked

In [6]:
dcon.shape

(91282, 91282)

In [14]:
# Now we are dealing with sparse vectors. Cosine similarity is used as affinity metric
#aff = 1 - pairwise_distances(dcon, metric = 'cosine')
aff = (pairwise_distances_chunked(dcon, metric = 'cosine'))
#aff = cosine_similarity(dcon)

In [ ]:
aff_list = list(aff)

In [ ]:
aff_np = np.array(aff_list)

In [12]:
aff = 1-aff

In [14]:
# Save affinity matrix
np.save('cosine_affinity.npy', aff)

Human Connectivity Embedding

In [10]:

# Load affinitity matrix
aff = np.load('cosine_affinity.npy')

In [11]:
aff.shape

(1470, 91282)

In [ ]:
'''
import sys
sys.path.append("../topography/utils_py/mapalign")
from mapalign import embed'''
import embed
emb, res = embed.compute_diffusion_map(aff, alpha = 0.5)

In [11]:
aff.shape

(1470, 91282)

In [ ]:

# Save results
np.save('gradient_data/embedded/embedding_dense_emb.npy', emb)
np.save('gradient_data/embedded/embedding_dense_res.npy', res)

In [ ]:
import numpy as np
res = np.load('gradient_data/embedded/embedding_dense_res.npy').item()
a = [res['vectors'][:,i]/ res['vectors'][:,0] for i in range(302)]
emb = np.array(a)[1:,:].T
len(emb)

Export to cifti space

In [ ]:

import nibabel as nib
import numpy as np

In [ ]:
res = nib.load('gradient_data/templates/hcp.tmp.lh.dscalar.nii').data
cortL = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
res = nib.load('gradient_data/templates/hcp.tmp.rh.dscalar.nii').data
cortR = np.squeeze(np.array(np.where(res != 0)[0], dtype=np.int32))
cortLen = len(cortL) + len(cortR)
del res
# save out cortR and cortL in gradient_data/templates/

In [ ]:
emb = np.load('gradient_data/embedded/embedding_dense_emb.npy')

In [ ]:
tmp = nib.nifti2.load('gradient_data/templates/100307_tfMRI_MOTOR_level2_hp200_s2.dscalar.nii')
tmp_cifti = nib.cifti.load('gradient_data/templates/100307_tfMRI_MOTOR_level2_hp200_s2.dscalar.nii')
data = tmp_cifti.data * 0
data[0:10,:len(emb)] = np.reshape(emb.T, [1, 1, 1, 1] + list(emb.T.shape))
tmp2 = nib.nifti2.Nifti2Image(data, tmp.get_affine())
tmp4 = nib.nifti2.create_cifti_image(tmp2, tmp_cifti.header.to_xml(), np.array(3006, dtype=np.int32))
tmp4.to_filename('gradient_data/embedded/ciftis/hcp.embed.dscalar.nii')